In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from model_utils import UniversalCarDataset, get_transforms  # Import your tools

# ==========================================
# 1. CONFIGURATION
# ==========================================
# Update this path to where your `prepare_data.py` saved the CSVs
DATA_DIR = "ready_data_splits" 
MODEL_SAVE_PATH = "resnet50_stanford_cars.pth"
BATCH_SIZE = 32
NUM_CLASSES = 196
NUM_EPOCHS = 15

# ==========================================
# 2. MODEL ARCHITECTURE
# ==========================================
def get_resnet_model(num_classes=196):
    # Load Pre-trained ResNet50
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    # --- STRATEGY: FINE-TUNING ---
    # 1. Freeze the early layers (generic features like lines/edges)
    for param in model.parameters():
        param.requires_grad = False
        
    # 2. Unfreeze the last TWO blocks for better feature learning (Optional but recommended)
    for param in model.layer3.parameters():
        param.requires_grad = True
    for param in model.layer4.parameters():
        param.requires_grad = True
        
    # 3. Replace the Head with Higher Dropout
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 1024), # Add an intermediate layer
        nn.ReLU(),
        nn.Dropout(0.5),              # INCREASED from 0.3 to 0.5
        nn.Linear(1024, num_classes)
    )

    return model

# ==========================================
# 3. TRAINING SKELETON
# ==========================================

if __name__ == "__main__":
    
    # Check if splits exist
    if not os.path.exists(DATA_DIR):
        print(f"Error: Data directory '{DATA_DIR}' not found.")
        print("Please run 'prepare_data.py' first to generate splits.")
    else:
        print("Initializing Standardized Pipeline...")
        
        try:
            # 1. Get Standard Transforms
            tfms = get_transforms(img_size=(224, 224))

            # 2. Load the Saved Splits
            print(f"Loading training data from {DATA_DIR}/train_split.csv...")
            train_ds = UniversalCarDataset(f"{DATA_DIR}/train_split.csv", transform=tfms['train'])
            
            print(f"Loading validation data from {DATA_DIR}/val_split.csv...")
            val_ds = UniversalCarDataset(f"{DATA_DIR}/val_split.csv", transform=tfms['val'])

            # 3. Create Loaders
            train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
            val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
            
            print("Initializing ResNet50...")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = get_resnet_model(num_classes=NUM_CLASSES).to(device)
            
            # Hyperparameters
            criterion = nn.CrossEntropyLoss()
            # Add weight_decay=1e-4
            optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
            
            # Learning Rate Scheduler
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
            
            print(f"Starting Training on {device}...")
            
            for epoch in range(NUM_EPOCHS):
                model.train()
                running_loss = 0.0
                correct_predictions = 0 
                total_samples = 0
                
                for images, labels in train_dl:
                    images, labels = images.to(device), labels.to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    running_loss += loss.item()
                    
                    # Track Accuracy during training to see progress
                    _, predicted = torch.max(outputs, 1)
                    total_samples += labels.size(0)
                    correct_predictions += (predicted == labels).sum().item()

                epoch_loss = running_loss / len(train_dl)
                epoch_acc = 100 * correct_predictions / total_samples
                
                print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
                
                # Update LR based on training loss (or validation loss if we computed it per epoch)
                scheduler.step(epoch_loss)
                
            print("Training loop finished successfully!")
            print("Saving model...")
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"Model saved as '{MODEL_SAVE_PATH}'")

        except Exception as e:
            print("\n An error occurred during execution:")
            print(e)
            import traceback
            traceback.print_exc()

Initializing Standardized Pipeline...
Loading training data from ready_data_splits/train_split.csv...
Loading validation data from ready_data_splits/val_split.csv...
Initializing ResNet50...
Starting Training on cuda...
Epoch 1/15 | Loss: 5.1178 | Accuracy: 2.49%
Epoch 2/15 | Loss: 4.1305 | Accuracy: 10.19%
Epoch 3/15 | Loss: 3.1685 | Accuracy: 20.60%
Epoch 4/15 | Loss: 2.4130 | Accuracy: 34.95%
Epoch 5/15 | Loss: 1.8673 | Accuracy: 47.08%
Epoch 6/15 | Loss: 1.4298 | Accuracy: 59.55%
Epoch 7/15 | Loss: 1.1141 | Accuracy: 67.58%
Epoch 8/15 | Loss: 0.9054 | Accuracy: 73.31%
Epoch 9/15 | Loss: 0.7432 | Accuracy: 78.17%
Epoch 10/15 | Loss: 0.6267 | Accuracy: 81.43%
Epoch 11/15 | Loss: 0.5394 | Accuracy: 84.66%
Epoch 12/15 | Loss: 0.4202 | Accuracy: 87.66%
Epoch 13/15 | Loss: 0.3788 | Accuracy: 89.13%
Epoch 14/15 | Loss: 0.3380 | Accuracy: 90.04%
Epoch 15/15 | Loss: 0.2742 | Accuracy: 92.35%
Training loop finished successfully!
Saving model...
Model saved as 'resnet50_stanford_cars.pth'
